In [3]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from tensorflow.keras.datasets.fashion_mnist import load_data

(train_images, train_labels), (test_images, test_labels) = load_data()
df = pd.read_csv('train.csv').drop(columns=['Id'])
dfPred = pd.read_csv('testX.csv')

4423680/4422102 [==============================] - 4s 1us/step


In [2]:
def f(x):
    pca = PCA(n_components = 50)
    ss = StandardScaler()
    X_train_pca, X_test_pca, y_train, y_test = train_test_split(pca.fit_transform(ss.fit_transform(df.iloc[:,1:].values)), df.iloc[:,0], test_size=0.1, random_state=x)
    X_pred = pca.transform(ss.transform(dfPred.iloc[:,1:].values))
    clf = SVC(random_state=x, C = 35)
    clf.fit(X_train_pca, y_train)
    return [x, '{0:.2f}%'.format(clf.score(X_test_pca, y_test)*100), clf.predict(X_pred)]

result = []
with Pool(40) as p:
    result = p.map(f, [*range(40)])

print(result[0])

[0, '89.35%', array([3, 4, 0, ..., 3, 2, 2])]


In [3]:
for i in range(40):
    print(result[i][1])

89.35%
89.77%
88.72%
89.17%
89.30%
88.97%
88.30%
89.63%
89.23%
90.10%
89.18%
88.98%
89.28%
88.92%
88.05%
88.88%
88.95%
88.95%
89.12%
89.30%
89.10%
88.68%
88.50%
89.42%
89.58%
89.47%
88.50%
88.53%
89.17%
89.80%
88.68%
89.33%
89.07%
89.72%
88.78%
89.82%
89.13%
88.85%
89.67%
89.53%


In [4]:
dfRes = dfPred[['Id']].copy()
for i in range(40):
    dfRes[result[i][0]] = result[i][2]
dfRes

,Id,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,0,3,3,4,3,3,4,3,3,3,...,3,3,3,4,3,3,3,3,3,4
1,1,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4,1,1,2,1,2,1,2,2,1,...,1,1,1,1,2,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,1,1,3,1,1,3,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9996,9996,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9997,9997,3,2,3,2,3,2,3,3,3,...,3,3,3,3,3,3,2,3,2,3
9998,9998,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [8]:
count = dfRes.apply(pd.Series.value_counts, axis=1)[[0, 1, 2, 3, 4]].fillna(0)

In [5]:
toFile = dfRes.drop(columns=['Id']).mode(axis=1)[[0]].astype(int).rename(columns={0: 'Label'})
toFile.index = toFile.index.rename('Id')
toFile.to_csv('SVMRandomCombine.csv', encoding='utf-8')

In [10]:
def g(x):
    pca = PCA(n_components = 50)
    ss = StandardScaler()
    X_train_pca, X_test_pca, y_train, y_test = train_test_split(pca.fit_transform(ss.fit_transform(df.iloc[:,1:].values)), df.iloc[:,0], test_size=0.1, random_state=x)
    X_pred = pca.transform(ss.transform(dfPred.iloc[:,1:].values))
    clf = RandomForestClassifier(max_features=0.5, min_samples_split=2, n_estimators=500, random_state=x)
    clf.fit(X_train_pca, y_train)
    return [x+40, '{0:.2f}%'.format(clf.score(X_test_pca, y_test)*100), clf.predict(X_pred)]

result2 = []
with Pool(40) as p:
    result2 = p.map(g, [*range(40)])

print(result2[0])

[40, '88.83%', array([3, 4, 0, ..., 2, 2, 2])]


In [11]:
for i in range(40):
    print(result2[i][1])

88.83%
89.48%
88.80%
88.82%
88.80%
88.95%
87.88%
88.88%
89.03%
89.82%
88.25%
89.17%
88.98%
88.73%
88.52%
88.50%
88.28%
88.90%
88.63%
88.93%
88.92%
87.98%
88.20%
88.98%
89.32%
89.50%
88.27%
88.38%
88.40%
89.00%
88.33%
88.55%
88.17%
88.98%
87.97%
89.98%
88.82%
88.92%
88.83%
88.88%


In [12]:
dfRes2 = dfPred[['Id']].copy()
for i in range(40):
    dfRes2[result2[i][0]] = result2[i][2]
dfRes2

,Id,40,41,42,43,44,45,46,47,48,...,70,71,72,73,74,75,76,77,78,79
0,0,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,1,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
9996,9996,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9997,9997,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
9998,9998,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [14]:
dfRes2.head(40)

,Id,40,41,42,43,44,45,46,47,48,...,70,71,72,73,74,75,76,77,78,79
0,0,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,1,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
5,5,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
8,8,1,1,1,1,2,1,2,1,1,...,1,2,1,1,1,1,1,1,2,1
9,9,2,2,3,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,3


In [15]:
toFile2 = dfRes2.drop(columns=['Id']).mode(axis=1)[[0]].astype(int).rename(columns={0: 'Label'})
toFile2.index = toFile2.index.rename('Id')
toFile2.to_csv('RFCombine.csv', encoding='utf-8')